## Does the loss go down?

### Mini-container:

In [1]:
### define utilities:
import numpy as np
import tensorflow as tf

np.random.seed(42)

def dual_opt(var_name_1, var_name_2, loss, optimizer):
    
    vars_1 = tf.get_collection(key = tf.GraphKeys.TRAINABLE_VARIABLES,
                                     scope= var_name_1)
    train_1 = optimizer.minimize(loss,var_list=vars_1)
        
    vars_2 = tf.get_collection(key = tf.GraphKeys.TRAINABLE_VARIABLES,
                                     scope = var_name_2)
    train_2 = optimizer.minimize(loss,var_list=vars_2)
    
    return tf.group(train_1, train_2)

def action_states(env,agent,actions):

    ss_ = np.concatenate((env.state_seq[env.iter-agent.horizon-1],env.state_seq[env.iter-1])).reshape((1,4))
    S = np.repeat(ss_,agent.horizon,axis=0)
            
    return np.concatenate((actions,S),axis=1)

### define environment:
class square_env:
    def __init__(self,duration,radius,dimension):
        if 2*radius > dimension:
            raise Warning("diameter can't exceed dimensions")
        self.R = radius # radius of agent
        self.dimension = dimension # LxW of the square world
        self.eps = radius/100
        self.lower_limit, self.upper_limit = self.R+self.eps, self.dimension-self.R-self.eps
        self.iter = 0 # current iteration
        self.duration = duration # maximum duration of agent in environment
        self.state_seq = np.zeros((self.duration,2))
                
    def random_initialisation(self):
        # method for initialisation: 
            
        self.state_seq[self.iter][0] = np.random.uniform(self.lower_limit, self.upper_limit)
        self.state_seq[self.iter][1] = np.random.uniform(self.lower_limit, self.upper_limit)
        
        self.iter = 1
        
    def boundary_conditions(self):
                
        #boundary conditions:
        cond_X = (self.state_seq[self.iter-1][0] >= self.lower_limit)*(self.state_seq[self.iter-1][0] <= self.upper_limit)
        cond_Y = (self.state_seq[self.iter-1][1] >= self.lower_limit)*(self.state_seq[self.iter-1][1] <= self.upper_limit)

        return cond_X*cond_Y
            
    def step(self, action):
                
        self.state_seq[self.iter] = self.state_seq[self.iter-1] + action
        
        #return to previous state if boundary conditions are not satisfied:
        if self.boundary_conditions() == 0:
            self.state_seq[self.iter] -= action
            
        self.iter += 1
            
        if self.iter > self.duration:
            raise Exception("Game over!")            
            
            
    def env_response(self,actions,horizon):
        # update the environment
        
        for i in range(1,horizon):
            self.step(actions[i])
        
        
    def reset(self):
        """
        Return to the initial conditions. 
        """
        self.state_seq = np.zeros((self.duration,2))
        self.iter = 0
        
        
### define agent:
import tensorflow_probability as tfp

class agent_cognition:
    
    """
        An agent that reasons using a measure of empowerment. 
        Here we assume that env refers to an initialised environment class. 
    """
    
    def __init__(self,planning_horizon,sess,seed, bound):
        self.sess = sess
        self.seed = seed
        self.horizon = planning_horizon        
        self.bound = bound
        
        self.current_state = tf.placeholder(tf.float32, [None, 2])
        self.source_action = tf.placeholder(tf.float32, [None, 2])
        # define a placeholder for beta values in the squared loss:
        self.beta = tf.placeholder(tf.float32, [None, 1])
        
        ## define a placeholder for the dropout value:
        self.prob = tf.placeholder_with_default(1.0, shape=(),name='prob')
        
        ## define a placeholder for the learning rate:
        self.lr = tf.placeholder(tf.float32, shape = [],name='lr')
        
        ## define empowerment critic:
        self.emp = self.empowerment_critic()
                
        ## define source:
        self.source_input_n = tf.placeholder(tf.float32, [None, 4])
        self.src_mu, self.src_log_sigma = self.source_dist_n()
        self.src_dist = tfp.distributions.MultivariateNormalDiag(self.src_mu, \
                                                             tf.exp(self.src_log_sigma))
                            
        self.log_src = self.src_dist.log_prob(self.source_action)
        
        
        ## define decoder parameters and log probability:
        self.decoder_input_n = tf.placeholder(tf.float32, [None, 6])
        self.decoder_mu, self.decoder_log_sigma = self.decoder_dist_n()
        
        self.decoder_dist = tfp.distributions.MultivariateNormalDiag(self.decoder_mu, \
                                                             tf.exp(self.decoder_log_sigma))
        
        self.log_decoder = self.decoder_dist.log_prob(self.source_action)
        
        ## define losses:
        self.decoder_loss = tf.reduce_mean(-1.0*self.decoder_dist.log_prob(self.source_action))
        
        self.squared_loss = tf.reduce_mean(tf.square(self.beta*self.log_decoder-self.emp-self.log_src))
        
        ### define the optimisers:
        self.fast_optimizer = tf.train.AdagradOptimizer(self.lr,name='ada_1')
        self.slow_optimizer = tf.train.AdagradOptimizer(self.lr,name='ada_2')
        
        self.train_decoder = self.fast_optimizer.minimize(self.decoder_loss)
        
        ### define a dual optimizatio method for critic and source:
        self.train_critic_and_source = dual_opt("critic", "source", self.squared_loss,\
                                                self.slow_optimizer)
        
    
        self.init_g = tf.global_variables_initializer() 
    
    def init_weights(self,shape,var_name):
        """
            Xavier initialisation of neural networks
        """
        initializer = tf.contrib.layers.xavier_initializer()
        return tf.Variable(initializer(shape),name = var_name)
        
    def two_layer_net(self, X, w_h, w_h2, w_o,bias_1, bias_2):
        """
            A generic method for creating two-layer networks
            
            input: weights
            output: neural network
        """
        
        h = tf.nn.elu(tf.add(tf.matmul(X, w_h),bias_1))
        drop_1 = tf.nn.dropout(h, self.prob)
        
        h2 = tf.nn.elu(tf.add(tf.matmul(drop_1, w_h2),bias_2))
        drop_2 = tf.nn.dropout(h2, self.prob)
        
        return tf.matmul(drop_2, w_o)
    
    def empowerment_critic(self):
        """
        This function provides a cheap approximation to empowerment
        upon convergence of the training algorithm. Given that the 
        mutual information is non-negative this function must only
        give non-negative output. 
        
        input: state
        output: empowerment estimate
        """
        
        #with tf.variable_scope("critic",reuse=tf.AUTO_REUSE):
        with tf.variable_scope("critic"):
            
            tf.set_random_seed(self.seed)
    
            w_h = self.init_weights([2,500],"w_h")
            w_h2 = self.init_weights([500,300],"w_h2")
            w_o = self.init_weights([300,1],"w_o")
            
            ### bias terms:
            bias_1 = self.init_weights([500],"bias_1")
            bias_2 = self.init_weights([300],"bias_2")
            bias_3 = self.init_weights([1],"bias_3")
                
            h = tf.nn.elu(tf.add(tf.matmul(self.current_state, w_h),bias_1))
            h2 = tf.nn.elu(tf.add(tf.matmul(h, w_h2),bias_2))
        
        return tf.nn.elu(tf.add(tf.matmul(h2, w_o),bias_3))
        
        
    def source_dist_n(self):
        
        """
            This is the per-action source distribution, also known as the 
            exploration distribution. 
        """
        
        #with tf.variable_scope("source",reuse=tf.AUTO_REUSE):
        with tf.variable_scope("source"):
                               
            tf.set_random_seed(self.seed)
            
            W_h = self.init_weights([4,300],"W_h")
            W_h2 = self.init_weights([300,100],"W_h2")
            W_o = self.init_weights([100,10],"W_o")
            
            # define bias terms:
            bias_1 = self.init_weights([300],"bias_1")
            bias_2 = self.init_weights([100],"bias_2")
            
            ## two-layer network:
            h = tf.nn.elu(tf.add(tf.matmul(self.source_input_n, W_h),bias_1))
            drop_1 = tf.nn.dropout(h, self.prob)
        
            h2 = tf.nn.elu(tf.add(tf.matmul(drop_1, W_h2),bias_2))
            drop_2 = tf.nn.dropout(h2, self.prob)
            
            Tau = tf.matmul(drop_2, W_o)
                                    
            W_mu = self.init_weights([10,2],"W_mu")
            W_sigma = self.init_weights([10,2],"W_sigma")
            
            mu = tf.matmul(Tau,W_mu)
            log_sigma = tf.multiply(tf.nn.tanh(tf.matmul(Tau,W_sigma)),self.bound)
            
        
        return mu, log_sigma
    
    
    def sampler(self,mu,log_sigma):
                        
        return np.random.normal(mu,np.exp(log_sigma))   
    
    def random_actions(self):
        """
            This baseline is used as a drop in replacement for the source at the
            early stages of learning and to check that the source isn't completely useless. 
        """
        
        return np.random.normal(0,self.bound,size = (self.horizon,2))
        
    
    def source_actions(self,state):
        
        actions = np.zeros((self.horizon,2))
        
        ### add a zero action to the state:
        AS_0 = np.concatenate((np.zeros(2),state))
        
        mu, log_sigma = self.sess.run([self.src_mu,self.src_log_sigma], feed_dict={ self.source_input_n: AS_0.reshape((1,4))})
                                                
        for i in range(1,self.horizon):
                        
            AS_n = np.concatenate((actions[i-1],state))
            
            mu, log_sigma = self.sess.run([self.src_mu,self.src_log_sigma], feed_dict={ self.source_input_n: AS_n.reshape((1,4))})
                        
            actions[i] = self.sampler(mu, log_sigma)
                    
        return actions
        
    def decoder_dist_n(self): 
        
        """
            This is the per-action decoder, also known as the 
            planning distribution. 
        """
        
        #with tf.variable_scope("decoder",reuse=tf.AUTO_REUSE):
        with tf.variable_scope("decoder"):
            
            tf.set_random_seed(self.seed)
                        
            W_h = self.init_weights([6,300],"W_h")
            W_h2 = self.init_weights([300,100],"W_h2")
            W_o = self.init_weights([100,10],"W_o")
            
            # define bias terms:
            bias_1 = self.init_weights([300],"bias_1")
            bias_2 = self.init_weights([100],"bias_2")
            
            ## two-layer network:
            h = tf.nn.elu(tf.add(tf.matmul(self.decoder_input_n, W_h),bias_1))
            h2 = tf.nn.elu(tf.add(tf.matmul(h, W_h2),bias_2))
            
            Tau = tf.matmul(h2, W_o)
                        
            W_mu = self.init_weights([10,2],"W_mu")
            W_sigma = self.init_weights([10,2],"W_sigma")
            
            mu = tf.matmul(Tau,W_mu)
            log_sigma = tf.multiply(tf.nn.tanh(tf.matmul(Tau,W_sigma)),self.bound)
                    
            
        return mu, log_sigma
    
    def decoder_actions(self,ss_):
        
        actions = np.zeros((self.horizon,2))
        
        ### add a zero action to the state:
        SS_0 = np.concatenate((np.zeros(2),ss_))
        
        mu, log_sigma = self.sess.run([self.decoder_mu,self.decoder_log_sigma], feed_dict={ self.decoder_input_n: SS_0.reshape((1,6))})
                                                
        for i in range(1,self.horizon):
                        
            SS_n = np.concatenate((actions[i-1],ss_))
    
            mu, log_sigma = self.sess.run([self.decoder_mu,self.decoder_log_sigma], feed_dict={ self.decoder_input_n: SS_n.reshape((1,6))})
                                
            actions[i] = self.sampler(mu, log_sigma)
                    
        return actions
    
    def mean_decoder_actions(self,ss_):
        
        actions, sigmas = np.zeros((self.horizon,2)), np.zeros((self.horizon,2))
        
        ### add a zero action to the state:
        SS_0 = np.concatenate((np.zeros(2),ss_))
        
        mu, log_sigma = self.sess.run([self.decoder_mu,self.decoder_log_sigma], feed_dict={ self.decoder_input_n: SS_0.reshape((1,6))})
                                                
        for i in range(1,self.horizon):
                        
            SS_n = np.concatenate((actions[i-1],ss_))
    
            mu, log_sigma = self.sess.run([self.decoder_mu,self.decoder_log_sigma], feed_dict={ self.decoder_input_n: SS_n.reshape((1,6))})
                                
            actions[i], sigmas[i] = mu, np.exp(log_sigma)
                    
        return actions, sigmas
                    
        return actions, sigmas

### 1. Define experiment:

In [2]:
%matplotlib inline
## set random seed:
tf.set_random_seed(42)

prob = 0.8

def training(seed, batch_size, lr,iters, horizon,bound):
    # define epoch counter:
    count = 0
    
    # define environment:    
    env = square_env(duration=horizon,radius=0.5,dimension=2*horizon*0.5)
        
    with tf.Session() as sess:
                
        A = agent_cognition(horizon,sess,seed,bound)   
        
        ### define beta schedule:
        betas = 1./np.array([min(0.001 + i/iters,1) for i in range(iters)])
                
        ## define inverse probability:
        inverse_prob = betas
        
        ### initialise the variables:
        sess.run(A.init_g)
        
        squared_losses, decoder_losses = np.zeros(iters), np.zeros(iters)
        
        for count in range(iters):
            
            ## reset the environment:
            env.reset()
            env.random_initialisation()
            
            mini_batch = np.zeros((batch_size*horizon,6))
            
            ## define mean and variance of environment:
            mu = env.dimension/2.0 - 0.5 ## mean of U(R,dimension-R)
            sigma = ((2*mu)**2)/12 ## variance of U(R,dimension-R)
            
            if count % 100 == 0:
                print(count)
            
            ### train our agent on a minibatch of recent experience:
            for i in range(batch_size):
                
                env.iter = 0
                                            
                if np.random.rand() > 1/inverse_prob[count]:
                    actions = A.random_actions()
                else:
                    normalised_state = (env.state_seq[env.iter]-mu)/sigma
                    
                    actions = A.source_actions(normalised_state)
                    
                env.iter += 1
                        
                ## get responses from the environment:
                env.env_response(actions,A.horizon)
                                    
                ## group actions, initial state, and final state:                        
                axx_ = action_states(env,A,actions)
                
                mini_batch[horizon*i:horizon*(i+1)] = axx_
            
            ## normalise the state representations:
            mini_batch[:,2:6] = (mini_batch[:,2:6] - mu)/sigma
                
            train_feed_1 = {A.decoder_input_n : mini_batch,A.source_action : mini_batch[:,0:2],\
                            A.prob : prob,A.lr:lr}
            
            sess.run(A.train_decoder,feed_dict = train_feed_1)
                
            # train source and critic:
            train_feed_2 = {A.beta: betas[count].reshape((1,1)), A.current_state: mini_batch[:,2:4],\
                            A.decoder_input_n : mini_batch, A.source_input_n : mini_batch[:,0:4], \
                            A.source_action : mini_batch[:,0:2],
                            A.prob : prob,A.lr:lr}
            
            sess.run(A.train_critic_and_source,feed_dict = train_feed_2)
                
            squared_losses[count] = sess.run(A.squared_loss,feed_dict = train_feed_2)
            decoder_losses[count] = sess.run(A.decoder_loss,feed_dict = train_feed_1)        
            
        return decoder_losses, squared_losses

### 2. Run first experiment:

In [3]:
horizon = 3
seed = [42,43]
bound = 1.0
iters = 10000 
batch_size = 50
num_expts = 2

lr = 0.01

decoder_losses = np.zeros((num_expts,iters))
squared_losses = np.zeros((num_expts,iters))

for i in range(num_expts):
    decoder_losses[i], squared_losses[i] = training(seed[i], batch_size,lr, iters, horizon,bound)

KeyboardInterrupt: 

### 3.Visualizing the relationship:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,10))
plt.style.use('ggplot')

f, axarr = plt.subplots(4, sharex=True,figsize=(20,10))
#plt.figure(figsize=(10,10))
axarr[0].set_title("first decoder loss")
axarr[0].plot(decoder_losses[0],'steelblue')

axarr[1].set_title("second decoder loss")
axarr[1].plot(decoder_losses[1],'steelblue')

axarr[2].set_title("first squared loss")
axarr[2].plot(squared_losses[0],'crimson')

axarr[3].set_title("second squared loss")
axarr[3].plot(squared_losses[1],'crimson')

plt.show()

### 4. Analysis of the loss curves: 

It appears that the loss goes to zero very quickly and then stabilizes. 

In [ ]:
decoder_losses[0][10],decoder_losses[0][100],decoder_losses[0][1000],  decoder_losses[0][-1]

In [ ]:
squared_losses[0][10],squared_losses[0][100],squared_losses[0][1000], squared_losses[0][-1]

It looks like the squared loss is still decreasing and that the decoder loss has basically decreased a lot faster than the squared loss. A reasonable next experiment would be to double
the number of iterations and see whether the squared loss approaches 0. I reckon that the reason why the decoder loss goes down more quickly is that it involves a maximum likelihood involving only one non-linear function.

### 5. Second experiment: 

In [ ]:
horizon = 3
seed = [42,43]
bound = 1.0
iters = 20000 
batch_size = 50
num_expts = 2

lr_1, lr_2 = 0.01, 0.01

decoder_losses = np.zeros((num_expts,iters))
squared_losses = np.zeros((num_expts,iters))

for i in range(num_expts):
    decoder_losses[i], squared_losses[i] = training(seed[i], batch_size,lr_1, lr_2, iters, horizon,bound)

### 6. Visualize the loss curves: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,10))
plt.style.use('ggplot')

f, axarr = plt.subplots(4, sharex=True,figsize=(20,10))
#plt.figure(figsize=(10,10))
axarr[0].set_title("first decoder loss")
axarr[0].plot(decoder_losses[0],'steelblue')

axarr[1].set_title("second decoder loss")
axarr[1].plot(decoder_losses[1],'steelblue')

axarr[2].set_title("first squared loss")
axarr[2].plot(squared_losses[0],'crimson')

axarr[3].set_title("second squared loss")
axarr[3].plot(squared_losses[1],'crimson')

plt.show()

### 7. Analysis of the loss curves: 

In [ ]:
squared_losses[0][10],squared_losses[0][100],squared_losses[0][1000],squared_losses[0][10000], squared_losses[0][-1]

In [ ]:
decoder_losses[1][10],decoder_losses[1][100],decoder_losses[1][1000],decoder_losses[0][10000],  decoder_losses[1][-1]